# Step-0 · Simulating 1-D Heat Diffusion (Warm-Up)

Consider a 1-D insulated rod with no internal heat sources or sinks.
The heat equation, a foundational PDE in physics and engineering, describes how heat diffuses along this rod:

$$\frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2}$$

where:
 -  $u(x,t)$ is the temperature at position $x$ and time $t$,
 - $\kappa$ is the thermal diffusivity of the material [$m^2/s$].


## Discretizing the Problem

To solve this equation numerically, we use a finite-volume approach:
 - Divide the 1-D domain into $N$ cells of equal size $\Delta x$.
 - Let $u_i(t)$ be the average temperature in the i-th cell.

Each cell exchanges heat with its neighbors through fluxes at the cell interfaces.

### Fourier's Law for Heat Flux

For the heat equation, the flux at a cell interface is given by Fourier's law:

$$F_{i+1/2} = -\kappa \frac{u_{i+1} - u_i}{\Delta x}$$

where
 - $F_{i+1/2}$ is the heat flux at the interface between cells i and i+1.
 - Positive $F$ is defined as flowing to the right.

When storing fluxes in arrays:
 - We use the convention $F[i]$ to represent the flux at the interface between cells i-1 and i.

At the outermost interfaces, we apply Neumann boundary conditions:
 - Prescribed heat fluxes at the boundaries ($q_L$ and $q_R$).
 - Setting $q_L$ = $q_R$ = 0.0 models an insulated rod (no heat leaving or entering the domain).

### Explicit Euler Update

The time evolution of each cell’s temperature is computed using the explicit Euler method:

$$u_i^{n+1} = u_i^n + \Delta t \left( \frac{F_{i-1/2} - F_{i+1/2}}{\Delta x} \right)$$

where:
 - $u_i^n$ is the temperature in cell i at time step n,
 - $\Delta t$ is the time step size,
 - $F_{i-1/2}$ and $F_{i+1/2}$ are the heat fluxes at the left and right interfaces of cell i, respectively.


### Exercise - Implement the timestepping routine

Complete the following function that advances the temperature by nt steps
using the formulas above. Keep it pure (do not mutate the input array). 

In [7]:
def solve_heat_eqn(u, kappa, dx, dt, qL, qR, nt):
    """Numerically solve the 1D heat equation.

    Arguments
    ---------
    u : array,      The initial temperature distribution (C)
    kappa : float,  The thermal diffusivity (m^2/s).
    dx : float,     The spacing between grid points (m).
    dt : float,     The time step size (s).
    qL : float,     The heat flux at the left boundary (W/m^2).
    qR : float,     The heat flux at the right boundary (W/m^2).
    nt : int,       The number of time steps to simulate.

    Returns
    -------
    array,          The updated temperature distribution.
    """

    N = len(u)          # Number of grid points
    F = [0.0] * (N+1)   # Fluxes at interfaces

    for _ in range(nt):

        # Boundary conditions
        F[0] = qL
        F[-1] = qR

        # Fluxes at intermediate interfaces
        for i in range(1, N):
            pass # TODO: F[i]: ...

        # Update the temperature distribution
        for i in range(len(u)):
            pass # TODO: u[i]: ...

    return u

***Solution***

Below collapsed cell includes a possible solution. Compare it with your own implementation. One possible source of inconsistency could be in the way the indexing for the fluxes and temperature are handled. Recall, the fluxes F are defined at the interfaces between the grid points, while the temperature u is defined at the grid points themselves.


In [8]:
def solve_heat_eqn(u, kappa, dx, dt, qL, qR, nt):
    """Numerically solve the 1D heat equation.

    Arguments
    ---------
    u : array,      The initial temperature distribution (C)
    kappa : float,  The thermal diffusivity (m^2/s).
    dx : float,     The spacing between grid points (m).
    dt : float,     The time step size (s).
    qL : float,     The heat flux at the left boundary (W/m^2).
    qR : float,     The heat flux at the right boundary (W/m^2).
    nt : int,       The number of time steps to simulate.

    Returns
    -------
    array,          The updated temperature distribution.
    """

    N = len(u)          # Number of grid points
    F = [0.0] * (N+1)   # Fluxes at interfaces

    for _ in range(nt):

        # Boundary conditions
        F[0] = qL
        F[-1] = qR

        # Fluxes at intermediate interfaces
        for i in range(1, N):
            F[i] = -kappa * (u[i] - u[i-1]) / dx

        # Update the temperature distribution
        for i in range(len(u)):
            dudt = (F[i] - F[i+1]) / dx
            u[i] += dt * dudt

    return u

### Execute the function

Now, let's try running this function for one timestep with the following input temperature distibution: [0.0, 100.0, 0.0]. The expected output is a new array representing the temperature distribution after one time step. One would expect the middle value, which is greater than its neighbors to decrease given the heat diffusion.

In [9]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # spatial step size (m)
    dt = 0.01,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1
)

[0.1, 99.8, 0.1]

Now, let's re-run the function with longer time steps and see how the temperature distribution evolves.

In [10]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # length of each spatial cell (m)
    dt = 1.0,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1
)

[10.0, 80.0, 10.0]

As expected, the temperature drop is more pronounced with a larger time step. Now let's run the simulation for 1000 time steps instead of just 1.

In [11]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # length of each spatial cell (m)
    dt = 0.1,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1000
)

[33.33333333333137, 33.333333333337265, 33.33333333333137]

The result of the above function call should converge to the steady-state solution of the heat equation, i.e., the mean temperature of the three nodes [0.0, 100.0, 0.0], i.e., 33.33 °C everywhere.

## What we did

 - We implemented a simple function to simulate the 1-D heat equation
   with Neumann (no-flux) boundary conditions.

 - Our approach was instinctive: get something working quickly to capture
   the essential physics of the problem.

 - We didn’t explicitly think about the computational properties the code
   should satisfy, how to structure it for clarity, or how to handle edge cases.

- Any checks we made were after-the-fact sanity checks, 
  rather than planned from the start.

This “code-and-fix” style is common in scientific computing. It works for prototyping
but has serious drawbacks as complexity grows. Even in our small example, a single
function mixes boundary conditions, flux computations, the update of the prognostic
variable (temperature) and time stepping.

In real models, similar routines can grow to hundreds or thousands of lines,
becoming hard to understand, test, and maintain. Often, tests and reasoning
about correctness only happen long after the code is written (if at all).


## What We Can Do Better

 - Define desired behavior first: Think carefully about the properties
   and invariants the code should satisfy before writing it.

 - Modularize: Break the implementation into smaller, reusable functions
   with clear responsibilities.

 - Build in testing early: Write unit tests and sanity checks
   alongside the code, not as an afterthought.

 - Reason systematically: Use logical reasoning to understand and
   guarantee the behavior of our code, even for edge cases.


In the remainder of this tutorial, we’ll refactor this initial prototype and explore strategies to make it more robust, testable, and trustworthy, ultimately gaining deeper confidence in both the code and the science it supports.